In [1]:
import dataclasses
import functools

import dotenv
import litellm
import pydantic
from litellm.caching.caching import Cache
from pydantic import field_validator
from pydantic_core import PydanticCustomError

from effectful.handlers.llm import Template, Tool
from effectful.handlers.llm.completions import (
    LiteLLMProvider,
    RetryLLMHandler,
    call_assistant,
    call_tool,
    get_message_sequence,
)
from effectful.ops.semantics import NotHandled, fwd, handler

dotenv.load_dotenv()

provider = LiteLLMProvider()

## Interface

The `effectful.handlers.llm` module provides a simplified LLM interface that uses algebraic effects for modularity. The module interface consists of:

- A decorator `Template.define` which creates a prompt template from a callable. A template is an LLM-implemented function whose behavior is specified by a template string. When a template is called, an LLM is invoked to produce the specified behavior.
- A decorator `Tool.define` which exposes Python callables as tools that templates can call. Tool signatures and docstrings define the schema passed to the model.
- Structured output handling via `Encodable` (used internally by templates and tool calls) to serialize/deserialize Python types.
- LLM providers such as `LiteLLMProvider`, and reliability helpers like `RetryLLMHandler` and `ReplayLiteLLMProvider`, which can be composed with `handler(...)` to control execution.

## Prompt Templates

This template function writes (bad) poetry on a given theme. While difficult to implement in Python, an LLM can provide a reasonable implementation.

In [2]:
@Template.define
def limerick(theme: str) -> str:
    """Write a limerick on the theme of {theme}. Do not use any tools."""
    raise NotHandled

If we call the template with a provider interpretation installed, we get reasonable behavior. The LLM is nondeterministic by default, so calling the template twice with the same arguments gives us different results.

Templates are regular callables, so can be converted to operations with `defop` if we want to override the LLM implementation in some cases.

In [3]:
with handler(provider):
    print(limerick("fish"))
    print("-" * 40)
    print(limerick("fish"))

In the sea, a sly fish did dwell,
Who loved spinning tales it would tell.  
With scales that would gleam,  
It chased the big dream,  
To swim where the free dolphins swell.
----------------------------------------
In the ocean, where waters do swish,
Swam a cod with a shimmering wish.
He thought he could fly,
Leap and touch the sky,
But landed with a slippery splish.


If we want deterministic behavior, we can cache the template call. We can either cache it with the default `@functools.cache` or use LiteLLM's built-in cache by setting a cache backend and passing `caching=True` to the provider:

In [4]:
@functools.cache
@Template.define
def haiku(theme: str) -> str:
    """Write a haiku on the theme of {theme}. Do not use any tools."""
    raise NotHandled


@Template.define
def haiku_no_cache(theme: str) -> str:
    """Write a haiku on the theme of {theme}. Do not use any tools."""
    raise NotHandled


print()
with handler(provider):
    print(haiku("fish"))
    print("-" * 40)
    print(haiku("fish"))

print()
# Enable LiteLLM caching by setting a cache backend and enabling caching.
litellm.cache = Cache()
provider_cached = LiteLLMProvider(caching=True)
try:
    with handler(provider_cached):
        print(haiku_no_cache("fish2"))
        print("-" * 40)
        print(haiku_no_cache("fish2"))
finally:
    litellm.cache = None

print()
with handler(provider):
    print(haiku_no_cache("fish3"))
    print("-" * 40)
    print(haiku_no_cache("fish3"))



Silent river glides,  
Silver scales beneath the light,  
Whispers from the deep.
----------------------------------------
Silent river glides,  
Silver scales beneath the light,  
Whispers from the deep.

Silent waters dance,
Colorful koi swim below,
Nature's gentle art.
----------------------------------------
In water they glide,  
Scales shimmering like jewels,  
Silent waves they ride.

Silvery scales gleam,
In still waters, deep they glide— 
Fish dance in the stream.
----------------------------------------
Silent waters flow,
Colorful fish swim below,
Nature's gentle glow.


## Converting LLM Results to Python Objects

Type conversion is handled by `decode`. By default, primitive types are converted. `DecodeError` is raised if a response cannot be converted.

In [5]:
@Template.define
def primes(first_digit: int) -> int:
    """Give a prime number with {first_digit} as the first digit. Do not use any tools."""
    raise NotHandled


with handler(provider):
    assert type(primes(6)) is int

More complex types can be converted by providing handlers for `decode`. Callable synthesis is supported via `Encodable` and the evaluation providers in `effectful.handlers.llm.evaluation` (`UnsafeEvalProvider` or `RestrictedEvalProvider`), which enable parsing/compiling/executing synthesized code.

In [6]:
import inspect
from collections.abc import Callable

from effectful.handlers.llm.evaluation import UnsafeEvalProvider


@Template.define
def count_char(char: str) -> Callable[[str], int]:
    """Write a function which takes a string and counts the occurrances of '{char}'. Do not use any tools."""
    raise NotHandled


# Use UnsafeEvalProvider for simple examples; RestrictedEvalProvider may need extra globals.
with handler(provider), handler(UnsafeEvalProvider()):
    count_a = count_char("a")
    assert callable(count_a)
    assert count_a("banana") == 3
    assert count_a("cherry") == 0
    # Print the source code of the generated function
    print(inspect.getsource(count_a))

def count_a(input_string: str) -> int:
    """
    Count the occurrences of the character 'a' in the given string.

    Args:
    input_string (str): The string to be searched.

    Returns:
    int: The number of times 'a' appears in the string.
    """
    count: int = 0
    for char in input_string:
        if char == 'a':
            count += 1
    return count


## Tool Calling

`Operation`s defined in the lexical scope of a `Template` are automatically available for the LLM to call as tools. The description of these operations is inferred from their type annotations and docstrings.

Tool calls are mediated by a helper operation `tool_call`. Handling this operation allows tool use to be tracked or logged.

In [7]:
@Tool.define
def cities() -> list[str]:
    """Return a list of cities that can be passed to `weather`."""
    return ["Chicago", "New York", "Barcelona"]


@Tool.define
def weather(city: str) -> str:
    """Given a city name, return a description of the weather in that city."""
    status = {"Chicago": "cold", "New York": "wet", "Barcelona": "sunny"}
    return status.get(city, "unknown")


@Template.define  # cities and weather auto-captured from lexical scope
def vacation() -> str:
    """Use the provided tools to suggest a city that has good weather. Use only the `cities` and `weather` tools provided."""
    raise NotHandled


def log_tool_call(tool, *args, **kwargs):
    result = fwd()
    print(f"Tool call: {tool}(*{args}, **{kwargs}) -> {result}")
    return result


with handler(provider), handler({Tool.__apply__: log_tool_call}):
    print(vacation())

Tool call: cities(*(), **{}) -> ['Chicago', 'New York', 'Barcelona']
Tool call: weather(*('Chicago',), **{}) -> cold
Tool call: weather(*('New York',), **{}) -> wet
Tool call: weather(*('Barcelona',), **{}) -> sunny
Based on the current weather descriptions:

- **Chicago**: Cold
- **New York**: Wet
- **Barcelona**: Sunny

Barcelona has sunny weather, which is often considered good weather by many people, so it would be a great choice!


## Structured Output Generation

Constrained generation is used for any type that is convertible to a Pydantic model.

In [8]:
@dataclasses.dataclass
class KnockKnockJoke:
    whos_there: str
    punchline: str


@Template.define
def write_joke(theme: str) -> KnockKnockJoke:
    """Write a knock-knock joke on the theme of {theme}. Do not use any tools."""
    raise NotHandled


@Template.define
def rate_joke(joke: KnockKnockJoke) -> bool:
    """Decide if {joke} is funny or not. Do not use any tools."""
    raise NotHandled


def do_comedy():
    joke = write_joke("lizards")
    print("> You are onstage at a comedy club. You tell the following joke:")
    print(
        f"Knock knock.\nWho's there?\n{joke.whos_there}.\n{joke.whos_there} who?\n{joke.punchline}"
    )
    if rate_joke(joke):
        print("> The crowd laughs politely.")
    else:
        print("> The crowd stares in stony silence.")


with handler(provider):
    do_comedy()

> You are onstage at a comedy club. You tell the following joke:
Knock knock.
Who's there?
Iguana.
Iguana who?
Iguana come in, it's really hot out here!
> The crowd laughs politely.


### Logging LLM requests
To intercept messages being called on the lower-level, we can write a handler for `completion`:

In [9]:
from effectful.handlers.llm.completions import get_message_sequence


def log_llm(*args, **kwargs):
    result = fwd(*args, **kwargs)

    # Print all messages (to show injected instructions like error feedback)
    if args and isinstance(args[0], list):
        messages = args[0]
    else:
        messages = list(get_message_sequence().values())

    for msg in messages:
        content = msg.get("content", "")
        if isinstance(content, list):
            text = "\n".join(
                block["text"]
                for block in content
                if isinstance(block, dict) and block.get("type") == "text"
            )
        else:
            text = str(content)
        print(f"> {text}")

    message = result[0] if isinstance(result, tuple) else result
    final_content = message.get("content") or message.get("reasoning_content")
    if final_content is not None:
        print(final_content)
    return result


# Avoid cache
try:
    haiku.cache_clear()
except Exception:
    pass

# Put completion handler innermost so it has highest precedence during the call
with handler(provider), handler({call_assistant: log_llm}):
    _ = haiku("fish2")
    _ = limerick("fish")  # or use haiku("fish-2") to avoid cache

> Write a haiku on the theme of fish2. Do not use any tools.
> {"value":"Silver scales that gleam,\nGentle ripples kiss the stream—\nFish in liquid dream."}
{"value":"Silver scales that gleam,\nGentle ripples kiss the stream—\nFish in liquid dream."}
> Write a limerick on the theme of fish. Do not use any tools.
> {"value":"There once was a fish quite spry,\nWho leapt out to touch the sky,\nBut with a splash and a grin,\nBack to the waves with a spin,\nIt swam off with hopes ever high."}
{"value":"There once was a fish quite spry,\nWho leapt out to touch the sky,\nBut with a splash and a grin,\nBack to the waves with a spin,\nIt swam off with hopes ever high."}


## Template Composition

Templates defined in the lexical scope are also captured, enabling template composition. One template can use the result of another template in a pipeline:


In [14]:
# Sub-templates for different story styles
@Template.define
def story_with_moral(topic: str) -> str:
    """Write a short story about {topic} and end with a moral lesson. Do not use any tools."""
    raise NotHandled


@Template.define
def story_funny(topic: str) -> str:
    """Write a funny, humorous story about {topic}. Do not use any tools."""
    raise NotHandled


# Main orchestrator template - has access to sub-templates
@Template.define
def write_story(topic: str, style: str) -> str:
    """Write a story about {topic} in the style: {style}.
    Available styles: 'moral' for a story with a lesson, 'funny' for humor. Use story_funny for humor, story_with_moral for a story with a lesson."""
    raise NotHandled


# Verify sub-templates are captured in write_story's lexical context
assert story_with_moral in write_story.tools.values()
assert story_funny in write_story.tools.values()
print("Sub-templates available to write_story:", write_story.tools.keys())

with handler(provider):
    print("=== Story with moral ===")
    print(write_story("a curious cat", "moral"))
    print()
    print("=== Funny story ===")
    print(write_story("a curious cat", "funny"))

Sub-templates available to write_story: dict_keys(['limerick', 'haiku_no_cache', 'primes', 'count_char', 'cities', 'weather', 'vacation', 'write_joke', 'rate_joke', 'story_with_moral', 'story_funny', 'unstable_service', 'fetch_data', 'give_rating_for_movie'])
=== Story with moral ===
The curious cat, Misty, learned that while shiny objects may be appealing, the true value often lies in the adventures and memories created along the way. It's a lesson in appreciating the intangible riches of life.🌟

=== Funny story ===


---

Here's a funny story about Mr. Whiskers, the ever-curious cat, that led to laughter, mystery, and magical adventures for his entire village!


### Retrying LLM Requests
LLM calls can sometimes fail due to transient errors or produce invalid outputs. The `RetryLLMHandler` automatically retries failed template calls and can also surface tool/runtime errors as tool messages:

- `num_retries`: Maximum number of retry attempts (default: 3)
- `include_traceback`: When `True`, include traceback details in the error feedback (default: False)
- `catch_tool_errors`: Exception type(s) to catch during tool execution (default: `Exception`)


Example usage: having an unstable service that seldomly fail.

In [13]:
call_count = 0
REQUIRED_RETRIES = 3


@Tool.define
def unstable_service() -> str:
    """Fetch data from an unstable external service. May require retries."""
    global call_count
    call_count += 1
    if call_count < REQUIRED_RETRIES:
        raise ConnectionError(
            f"Service unavailable! Attempt {call_count}/{REQUIRED_RETRIES}. Please retry."
        )
    return "{ 'status': 'ok', 'data': [1, 2, 3] }"


@Template.define  # unstable_service auto-captured from lexical scope
def fetch_data() -> str:
    """Use the unstable_service tool to fetch data."""
    raise NotHandled

with handler(provider):
    try:
        result = fetch_data()
    except Exception as e:
        print(f"Error: {e}")

with handler(provider), handler(RetryLLMHandler(num_retries=3)):
    result = fetch_data()
    print(f"Result: {result}", "Retries:", call_count)

Error: Service unavailable! Attempt 1/3. Please retry.
Result: The data fetched from the unstable service is: [1, 2, 3]. Retries: 3


### Retrying with Validation Errors
As noted above, the `RetryHandler` can also be used to retry on runtime/validation error:

In [ ]:
@pydantic.dataclasses.dataclass
class Rating:
    score: int
    explanation: str

    @field_validator("score")
    @classmethod
    def check_score(cls, v):
        if v < 1 or v > 5:
            raise PydanticCustomError(
                "invalid_score",
                "score must be 1–5, got {v}",
                {"v": v},
            )
        return v

    @field_validator("explanation")
    @classmethod
    def check_explanation_contains_score(cls, v, info):
        score = info.data.get("score", None)
        if score is not None and str(score) not in v:
            raise PydanticCustomError(
                "invalid_explanation",
                "explanation must mention the score {score}, got '{explanation}'",
                {"score": score, "explanation": v},
            )
        return v


@Template.define
def give_rating_for_movie(movie_name: str) -> Rating:
    """Give a rating for {movie_name}. The explanation MUST include the numeric score. Do not use any tools."""
    raise NotHandled

with handler(provider):
    try:
        rating = give_rating_for_movie("Die Hard")
    except Exception as e:
        print(f"Error: {e}")

with handler(provider), handler(RetryLLMHandler(num_retries=3)):
    rating = give_rating_for_movie("Die Hard")
    print(f"Score: {rating.score}/5")
    print(f"Explanation: {rating.explanation}")

Score: 5/5
Explanation: Die Hard deserves a 5 out of 5. This film is a quintessential action movie, showcasing Bruce Willis in an iconic role that defined his career. Its gripping plot, high-stakes action scenes, and outstanding villain performance by Alan Rickman make it a timeless classic. These elements together ensure its place as a peak action movie experience, hence the perfect score.
